In [ ]:
import sys
from pathlib import Path
import torch
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import importlib

# 讓 Python 找得到 src 資料夾
# 假設 notebook 在專案根目錄 (跟 src 同層)
sys.path.append(str(Path.cwd()))

# 設定繪圖風格
sns.set_theme(style="whitegrid")
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'Microsoft JhengHei']
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 取得目前 notebook 的路徑 (例如 D:\NCKU\paper4\script)
current_dir = Path.cwd()

# 取得上一層目錄 (專案根目錄，例如 D:\NCKU\paper4)
project_root = current_dir.parent

# 把根目錄加入 Python 的搜尋路徑
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    print(f"已加入專案路徑: {project_root}")

# reload
to_delete = [m for m in sys.modules if m.startswith('src')]

for m in to_delete:
    del sys.modules[m]

print(f"已清理模組: {to_delete}")

import src.utils.data_loader as data_loader
import src.engine.trainer as trainer
import src.engine.evaluator as evaluator

# 硬體設定
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on: {DEVICE}")

In [ ]:
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# 全域設定
DATA_PATH = Path("../dataset/USD_TWD.csv")
HORIZON = 3
LOOKBACK = 65
EPOCHS = 100
LR = 0.001
SEED = 42
TREND_CNNEXPERT_KERNELSIZE = 7
SEASONAL_CNNEXPERT_KERNELSIZE = 3
SERIESDECOMPOSITION_KERNELSIZE = 7

# 設定隨機性 (Reproducibility)
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)


In [ ]:
if not DATA_PATH.exists():
    raise FileNotFoundError("找不到資料集！")

df = pd.read_csv(DATA_PATH)

# 載入資料
train_loader, test_loader, scalers, _, _, _, _ = data_loader.prepare_data(
    df, lookback=LOOKBACK, horizon=HORIZON
)

print(f"Data ready. Train batches: {len(train_loader)}, Test batches: {len(test_loader)}")

In [ ]:
ablation_experiments = [
    {
        "name": "Full Model (Baseline)",
        "params": {}  # 空字典 = 全部使用預設值 (全開)
    },
    # {
    #     "name": "w/o CNN",
    #     "params": {"use_cnn": False}  # 關閉 CNN
    # },
    {
        "name": "w/o Decomposition",
        "params": {"use_decomp": False}  # 關閉序列分解
    },
    # 如果你有做 Gate 機制也可以測
    # {
    #     "name": "w/o Gating",
    #     "params": {"use_gate": False}
    # }
]

In [ ]:
results = []

print(f"開始執行 {len(ablation_experiments)} 組消融實驗...\n")

for i, exp in enumerate(ablation_experiments):
    exp_name = exp["name"]
    exp_params = exp["params"]

    print(f"[{i + 1}/{len(ablation_experiments)}] Running: {exp_name}")
    print(f"   Params: {exp_params}")

    # 1. 重置種子
    set_seed(42)

    # 2. 訓練模型
    model = trainer.train_v11(
        train_loader=train_loader,
        test_loader=test_loader,
        device=DEVICE,
        horizon=HORIZON,
        num_epochs=EPOCHS,
        seq_len=LOOKBACK,
        lr=LR,
        trendCNNExpert_KernelSize=TREND_CNNEXPERT_KERNELSIZE,
        seasonalCNNExpert_KernelSize=SEASONAL_CNNEXPERT_KERNELSIZE,
        seriesDecomposition_KernelSize=SERIESDECOMPOSITION_KERNELSIZE,
        model_hyperparams=exp_params
    )

    # 3. 評估模型 (修正點：移除 scaler 參數)
    metrics = evaluator.evaluate_model(
        model=model,
        test_loader=test_loader,
        device=DEVICE,  # <--- 修正：只傳入 device 和 horizon
        horizon=HORIZON
    )

    # 4. 儲存結果
    record = {"Experiment": exp_name}
    record.update(metrics)
    results.append(record)

    # 修正點：改印 RMSE_Final
    rmse = metrics.get('RMSE_Final', 0.0)
    print(f"   Result: RMSE={rmse:.4f}\n")

print("所有實驗結束！")


In [ ]:
# 轉成 DataFrame
df_res = pd.DataFrame(results)

# 顯示表格
print("\n=== Ablation Study Results ===")
display(df_res)

# === 畫圖比較 1: RMSE (越低越好) ===
plt.figure(figsize=(10, 5))
# 修正點：y 改為 'RMSE_Final'
sns.barplot(data=df_res, x='Experiment', y='RMSE_Final', hue='Experiment', palette='viridis', legend=False)

plt.title(f"RMSE Comparison (Lower is Better)", fontsize=14)
plt.ylabel("RMSE")
plt.xlabel("Model Configuration")

for index, row in df_res.iterrows():
    plt.text(index, row['RMSE_Final'], f"{row['RMSE_Final']:.4f}", color='black', ha="center", va="bottom")

plt.tight_layout()
plt.show()

# -------------------------------
plt.figure(figsize=(10, 5))
sns.barplot(data=df_res, x='Experiment', y='Avg_Acc', hue='Experiment', palette='magma', legend=False)

plt.title(f"Average Direction Accuracy (Higher is Better)", fontsize=14)
plt.ylabel("Accuracy")
plt.xlabel("Model Configuration")

for index, row in df_res.iterrows():
    plt.text(index, row['Avg_Acc'], f"{row['Avg_Acc']:.4f}", color='black', ha="center", va="bottom")

plt.tight_layout()
plt.show()
